# Day 19: 物体検出（Sliding Window）

## Learning Objectives
- スライディングウィンドアルゴリズムを理解する
- 非最大値抑制（NMS）を実装する
- マルチスケール物体検出を実装する

## スライディングウィンドウ実装

物体検出の基本的な手法として、画像全体にウィンドウをスライドさせながら物体を検出します。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle

def sliding_window(image, window_size, step_size):
    """スライディングウィンドウを生成
    
    Args:
        image: 入力画像（2次元配列）
        window_size: ウィンドウサイズ (width, height)
        step_size: ステップサイズ (x_step, y_step)
    
    Returns:
        ウィンドウの位置と画像スライスのリスト
    """
    windows = []
    h, w = len(image), len(image[0])
    window_w, window_h = window_size
    step_x, step_y = step_size
    
    for y in range(0, h - window_h + 1, step_y):
        for x in range(0, w - window_w + 1, step_x):
            window = image[y:y+window_h, x:x+window_w]
            windows.append((x, y, window))
    
    return windows

# テスト用のダミー画像
test_image = np.random.rand(100, 100)
windows = sliding_window(test_image, (20, 20), (10, 10))

print(f"生成されたウィンドウ数: {len(windows)}")
print(f"最初のウィンドウ位置: {windows[0][:2]}")

In [ ]:
def visualize_sliding_window(image, window_size, step_size, num_windows_to_show=9):
    """スライディングウィンドウを可視化"""
    windows = sliding_window(image, window_size, step_size)
    
    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    axes = axes.flatten()
    
    for i in range(min(num_windows_to_show, len(windows))):
        x, y, window = windows[i]
        axes[i].imshow(window, cmap='gray')
        axes[i].set_title(f'Window {i}: ({x}, {y})')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# ウィンドウの可視化
visualize_sliding_window(test_image, (20, 20), (10, 10))

## 物体検出シミュレーション

実際に物体を含む画像を作成し、スライディングウィンドウによる検出をシミュレートします。

In [ ]:
def create_test_image_with_objects():
    """テスト用の物体付き画像を作成"""
    # 100x100の白い画像
    image = np.ones((100, 100)) * 255
    
    # 物体1: 20x20の黒い矩形
    image[20:40, 30:50] = 0
    
    # 物体2: 15x15の灰色の円
    center_y, center_x = 70, 70
    radius = 7
    for y in range(max(0, center_y-radius), min(100, center_y+radius+1)):
        for x in range(max(0, center_x-radius), min(100, center_x+radius+1)):
            if (x - center_x)**2 + (y - center_y)**2 <= radius**2:
                image[y, x] = 128
    
    return image

# テスト画像を作成
test_img_with_objects = create_test_image_with_objects()
plt.imshow(test_img_with_objects, cmap='gray')
plt.title('Test Image with Objects')
plt.axis('off')
plt.show()

In [ ]:
def simple_object_detector(window, threshold=0.1):
    """単純な物体検出器
    
    ウィンドウ内の暗いピクセルの割合が閾値を超えたら物体と判定
    """
    dark_pixels = np.sum(window < 200)  # 200以下のピクセルを「暗い」と判定
    total_pixels = window.size
    dark_ratio = dark_pixels / total_pixels
    
    return dark_ratio > threshold

def detect_objects_with_sliding_window(image, window_size, step_size, threshold=0.1):
    """スライディングウィンドウで物体を検出"""
    windows = sliding_window(image, window_size, step_size)
    detections = []
    
    for x, y, window in windows:
        if simple_object_detector(window, threshold):
            detections.append((x, y, window_size[0], window_size[1]))
    
    return detections

# 物体検出を実行
detections = detect_objects_with_sliding_window(
    test_img_with_objects, 
    window_size=(15, 15), 
    step_size=5,
    threshold=0.1
)

print(f"検出された物体数: {len(detections)}")
for i, (x, y, w, h) in enumerate(detections):
    print(f"物体 {i+1}: 位置=({x}, {y}), サイズ=({w}, {h})")

def visualize_detections(image, detections):
    """検出結果を可視化"""
    plt.figure(figsize=(10, 8))
    plt.imshow(image, cmap='gray')
    
    # 検出枠を描画
    for x, y, w, h in detections:
        rect = Rectangle((x, y), w, h, linewidth=2, edgecolor='red', facecolor='none')
        plt.gca().add_patch(rect)
    
    plt.title('Object Detection Results')
    plt.axis('off')
    plt.show()

# 検出結果の可視化
visualize_detections(test_img_with_objects, detections)

## 非最大値抑制（NMS）

複数の検出枠が重複している場合、最もスコアの高いものだけを残します。

In [ ]:
def calculate_iou(box1, box2):
    """2つのバウンディングボックスのIOUを計算"""
    # box = (x, y, w, h)
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    # 交差部分の座標
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1 + w1, x2 + w2)
    y_bottom = min(y1 + h1, y2 + h2)
    
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - intersection_area
    
    return intersection_area / union_area

def non_max_suppression(detections, iou_threshold=0.5):
    """非最大値抑制を実行"""
    if not detections:
        return []
    
    # スコア順にソート（今回はランダムなスコアを割り当て）
    detections_with_scores = []
    for i, det in enumerate(detections):
        # ダミーのスコア：面積が大きいほど高いスコア
        score = det[2] * det[3]
        detections_with_scores.append((score, i, det))
    
    detections_with_scores.sort(reverse=True)
    
    suppressed = []
    used = [False] * len(detections)
    
    while detections_with_scores:
        best_score, best_idx, best_det = detections_with_scores.pop(0)
        if used[best_idx]:
            continue
        
        suppressed.append(best_det)
        used[best_idx] = True
        
        # IOUが閾値より大きい検出を抑制
        for score, idx, det in detections_with_scores:
            if not used[idx] and calculate_iou(best_det, det) > iou_threshold:
                used[idx] = True
    
    return suppressed

# NMSの実行
detections_with_overlap = [
    (25, 25, 20, 20),  # 物体1
    (30, 30, 20, 20),  # 物体1と重複
    (70, 70, 15, 15),  # 物体2
    (75, 75, 15, 15),  # 物体2と重複
    (10, 10, 10, 10),  # 偽陽性
]

nms_result = non_max_suppression(detections_with_overlap, iou_threshold=0.5)
print(f"NMS後の検出数: {len(nms_result)}")
for i, (x, y, w, h) in enumerate(nms_result):
    print(f"選択された検出 {i+1}: 位置=({x}, {y}), サイズ=({w}, {h})")

def visualize_nms(image, original_detections, nms_result):
    """NMSの前後を比較表示"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # NMS前
    ax1.imshow(image, cmap='gray')
    for x, y, w, h in original_detections:
        rect = Rectangle((x, y), w, h, linewidth=1, edgecolor='blue', facecolor='none')
        ax1.add_patch(rect)
    ax1.set_title('Before NMS')
    ax1.axis('off')
    
    # NMS後
    ax2.imshow(image, cmap='gray')
    for x, y, w, h in nms_result:
        rect = Rectangle((x, y), w, h, linewidth=2, edgecolor='red', facecolor='none')
        ax2.add_patch(rect)
    ax2.set_title('After NMS')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

# NMSの可視化
visualize_nms(test_img_with_objects, detections, nms_result)

## マルチスケール物体検出

異なるサイズの物体を検出するために、複数のスケールでスライディングウィンドウを実行します。

In [ ]:
def multi_scale_object_detection(image, scales, window_size, step_size, threshold=0.1):
    """マルチスケール物体検出"""
    all_detections = []
    
    for scale in scales:
        # スケールに応じた画像サイズの変化
        scaled_size = int(window_size[0] * scale), int(window_size[1] * scale)
        
        # 画像をリサイズ（補間）
        h, w = len(image), len(image[0])
        scaled_h, scaled_w = int(h * scale), int(w * scale)
        
        # 簡易的なリサイズ（最近傍補間）
        scaled_image = np.zeros((scaled_h, scaled_w))
        for y in range(scaled_h):
            for x in range(scaled_w):
                orig_y = int(y / scale)
                orig_x = int(x / scale)
                if orig_y < h and orig_x < w:
                    scaled_image[y, x] = image[orig_y, orig_x]
        
        # スケーリングされた画像で検出
        scaled_detections = detect_objects_with_sliding_window(
            scaled_image, 
            scaled_size, 
            step_size,
            threshold
        )
        
        # 元のスケールに戻す
        original_scale_detections = []
        for x, y, w, h in scaled_detections:
            orig_x = int(x / scale)
            orig_y = int(y / scale)
            orig_w = int(w / scale)
            orig_h = int(h / scale)
            original_scale_detections.append((orig_x, orig_y, orig_w, orig_h))
        
        all_detections.extend(original_scale_detections)
        
        print(f"スケール {scale:.2f}: {len(scaled_detections)}個検出")
    
    # すべての検出に対してNMSを実行
    final_detections = non_max_suppression(all_detections, iou_threshold=0.3)
    
    return final_detections

# マルチスケール検出
scales = [0.5, 1.0, 1.5]  # 50%, 100%, 150%のスケール
multi_scale_detections = multi_scale_object_detection(
    test_img_with_objects,
    scales,
    window_size=(15, 15),
    step_size=5,
    threshold=0.1
)

print(f"\nマルチスケール検出結果: {len(multi_scale_detections)}個の物体を検出")

# マルチスケール検出結果の可視化
visualize_detections(test_img_with_objects, multi_scale_detections)

## まとめと次のステップ

### 習得したスキル:
- [x] スライディングウィンドウアルゴリズムを実装した
- [x] 非最大値抑制（NMS）を実装した
- [x] マルチスケール物体検出を実装した
- [x] IOU（Intersection over Union）を理解した

### 考察:
1. スライディングウィンドウは計算コストが高いが、シンプルな物体検出方法です
2. NMSは重複する検出を削除するために不可欠です
3. マルチスケール検出は、異なるサイズの物体を検出する効果的な方法です

### 次のステップ:
- 特徴量抽出（HOGなど）を組み合わせたより高度な物体検出
- 深層学習による物体検出（YOLO, Faster R-CNNなど）